In [1]:
%reset -f

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.api import VAR
from scipy.stats import linregress

In [3]:
df_r = pd.read_excel('/Users/cristianosalvagnin/Documents/GitHub/EUA_Volatility_DII_p/Data/Dataset_EUA_24_Daily_Returns.xlsx')
df_r = df_r.drop(columns=['Date'])

In [4]:
scaler = StandardScaler()
data_r_s = scaler.fit_transform(df_r.iloc[:,:])
df_r_s = pd.DataFrame(data_r_s, columns=df_r.columns[:])

In [5]:
labels = [
    'EUA', 'GPR', 'VSTOXX', 'UncEURUSD', 'UncEURJPY', 'UncEURGBP', 'UncEURCHF',
    'NatGas', 'ElecES', 'ElecDE', 'ElecFR', 'CoalFut', 'CuFut', 'Brent',
    'AgFut', 'Gold', 'EURUSD', 'EURJPY', 'EURGBP', 'EURCHF', 'WHNewEnergy',
    'BbgEnergy', 'SolCEA', 'ESTXElect', 'SEF EU50', 'LC100EU', 'MSCIEnrg',
    'ERIX', 'Euronext100', 'IBEX35', 'DAX', 'CAC', 'FTSEmib', 'Bund10y', 'Bond3m'
]

In [6]:
df_r_s.columns = labels
df_r.columns = labels

In [7]:
model = VAR(df_r_s)
var_model = model.fit(1, trend='n', verbose=True)  # 1 lag


In [ ]:
var_model.summary()

In [ ]:
causality_results = []

for var in df_r_s.columns:
    causality = var_model.test_causality('EUA', var, kind='f')
    
    causality_results.append({
        'Excluded Variable': var,
        'F-Statistic': causality.test_statistic,
        'p-value': causality.pvalue
    })

causality_results_df = pd.DataFrame(causality_results)

print(causality_results_df)

In [ ]:
causality_results_df = causality_results_df.sort_values(by='F-Statistic', ascending=False)
print(causality_results_df)

In [ ]:
eua_var_coef = abs(var_model.params['EUA'])
eua_var_coef = eua_var_coef.sort_values(ascending=False)
print(eua_var_coef)

In [ ]:
eua_var_coef.index = eua_var_coef.index.str.replace('L1.', '')
print(eua_var_coef)

In [ ]:
eua_var_coef = eua_var_coef.reset_index()
eua_var_coef.columns = ['Variable', 'VAR Coeff.']
print(eua_var_coef)

In [14]:
with open('/Users/cristianosalvagnin/Documents/GitHub/EUA_Volatility_DII_p/Data/weights_DII_E1_returns.p', 'rb') as file:
    DII_Rets_1 = pickle.load(file)

In [ ]:
df_DII_Rets_1 = pd.DataFrame(DII_Rets_1)
df_DII_Rets_1.columns = ['DII Coeff.']
print(df_DII_Rets_1)

In [ ]:
df_DII_Rets_1['Variable'] = labels
df_DII_Rets_1 = df_DII_Rets_1[['Variable', 'DII Coeff.']]
print(df_DII_Rets_1)

In [ ]:
df_DII_Rets_1 = df_DII_Rets_1.sort_values(by='DII Coeff.', ascending=False)
print(df_DII_Rets_1)

In [ ]:
merged_df = pd.merge(eua_var_coef, df_DII_Rets_1, on='Variable', how='inner')
print(merged_df)

In [ ]:
bold_vars = ["CoalFut", "CAC", "IBEX35", "ElecFR", "Euronext100"]

eua_var_coef["Variable"] = eua_var_coef["Variable"].apply(
    lambda x: f"$\\bf{{{x}}}$" if x in bold_vars else x
)

df_DII_Rets_1["Variable"] = df_DII_Rets_1["Variable"].apply(
    lambda x: f"$\\bf{{{x}}}$" if x in bold_vars else x
)

fig, axes = plt.subplots(1, 3, figsize=(16, 8), gridspec_kw={'width_ratios': [1, 1, 2]})

bar_kwargs = {"edgecolor": "black", "linewidth": 1.2}

axes[0].barh(
    eua_var_coef["Variable"], 
    eua_var_coef["VAR Coeff."], 
    color='tab:blue', 
    **bar_kwargs
)
axes[0].set_xlabel("VAR Coefficients", fontsize=16)
axes[0].invert_yaxis()
axes[0].grid(axis='x', linestyle='--', alpha=0.7)
axes[0].tick_params(axis='x', labelsize=14)
axes[0].tick_params(axis='y', labelsize=14)

axes[1].barh(
    df_DII_Rets_1["Variable"], 
    df_DII_Rets_1["DII Coeff."], 
    color='tab:orange', 
    **bar_kwargs
)
axes[1].set_xlabel("DII Coefficients", fontsize=16)
axes[1].invert_yaxis()
axes[1].grid(axis='x', linestyle='--', alpha=0.7)
axes[1].tick_params(axis='x', labelsize=14)
axes[1].tick_params(axis='y', labelsize=14)

axes[2].scatter(
    merged_df["VAR Coeff."], 
    merged_df["DII Coeff."], 
    color='tab:pink', 
    alpha=0.9, s=200, edgecolor='black'
)
axes[2].set_xlabel("VAR Coefficients", fontsize=16)
axes[2].set_ylabel("DII Coefficients", fontsize=16)
axes[2].grid(True, linestyle='--', alpha=0.7)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.tight_layout()
plt.show()


***

In [ ]:
causality_results_df = causality_results_df.drop(columns=['p-value'])
causality_results_df.columns = ['Variable', 'F-Statistic']
print(causality_results_df)

In [ ]:
with open('/Users/cristianosalvagnin/Documents/GitHub/EUA_Volatility_DII_p/Data/IG_DII_E1_returns.p', 'rb') as file:
    IG_Rets_1 = pickle.load(file)

In [ ]:
df_IG_Rets_1 = pd.DataFrame(IG_Rets_1)
df_IG_Rets_1.columns = ['IG Coeff.']
print(df_IG_Rets_1)

In [ ]:
df_IG_Rets_1['Variable'] = labels
df_IG_Rets_1 = df_IG_Rets_1[['Variable', 'IG Coeff.']]
print(df_IG_Rets_1)

In [ ]:
df_IG_Rets_1 = df_IG_Rets_1.sort_values(by='IG Coeff.', ascending=False)
print(df_IG_Rets_1)

In [ ]:
merged_df_2 = pd.merge(causality_results_df, df_IG_Rets_1, on='Variable', how='inner')
print(merged_df_2)

In [ ]:
bold_vars = ["IBEX35", "CoalFut", "Gold", "CuFut"]

causality_results_df["Variable"] = causality_results_df["Variable"].apply(
    lambda x: f"$\\bf{{{x}}}$" if x in bold_vars else x
)

df_IG_Rets_1["Variable"] = df_IG_Rets_1["Variable"].apply(
    lambda x: f"$\\bf{{{x}}}$" if x in bold_vars else x
)

fig, axes = plt.subplots(1, 3, figsize=(16, 8), gridspec_kw={'width_ratios': [1, 1, 2]})

bar_kwargs = {"edgecolor": "black", "linewidth": 1.2}

axes[0].barh(
    causality_results_df["Variable"], 
    causality_results_df["F-Statistic"], 
    color='tab:blue', 
    **bar_kwargs
)

axes[0].set_xlabel("F-Statistic", fontsize=16)
axes[0].invert_yaxis()
axes[0].grid(axis='x', linestyle='--', alpha=0.7)

axes[0].tick_params(axis='x', labelsize=14)
axes[0].tick_params(axis='y', labelsize=14)

axes[1].barh(
    df_IG_Rets_1["Variable"], 
    df_IG_Rets_1["IG Coeff."], 
    color='tab:orange', 
    **bar_kwargs
)

axes[1].set_xlabel("Imbalance Gain", fontsize=16)
axes[1].invert_yaxis()
axes[1].grid(axis='x', linestyle='--', alpha=0.7)

axes[1].tick_params(axis='x', labelsize=14)
axes[1].tick_params(axis='y', labelsize=14)

axes[2].scatter(
    merged_df_2["F-Statistic"], 
    merged_df_2["IG Coeff."], 
    color='tab:pink', 
    alpha=0.9, s=200, edgecolor='black'
)

axes[2].set_xlabel("F-Statistic", fontsize=16)
axes[2].set_ylabel("Imbalance Gain", fontsize=16)
axes[2].grid(True, linestyle='--', alpha=0.7)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.tight_layout()
plt.show()

***